# Image Classification Task 

- This task is working on 3 classes of cifar10 classes ( car = 1 , cat = 3 , dog = 5 ). 
- this was Bulit using  keras (based on TensorFlow package ). 

## Importing Packages

In [100]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np


### Making Configs variables

In [125]:
batch_size = 32
num_classes = 3
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

## Reading Dataset

* **CIFAR-10**

In [94]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = y_train.reshape(y_train.shape[0])  # somehow y_train comes as a 2D nx1 matrix
y_test = y_test.reshape(y_test.shape[0])

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'training samples)')
print(X_test.shape[0], 'validation samples)')

X_train shape: (50000, 3, 32, 32)
50000 training samples)
10000 validation samples)


### Data Manipulation

* Selecting 3 needed classes(`Dog`, `Cat`, and `Car`) of 10 total classes.

In [117]:
# Selecting classes in trainning set
specified_x_train = []
specified_y_train = []

for i in range(len(y_train)):
    if (y_train[i] == 1)  :
        specified_x_train.append(X_train[i])
        specified_y_train.append(0)
    elif (y_train[i] == 3):
        specified_x_train.append(X_train[i])
        specified_y_train.append(1)
    elif (y_train[i] == 5):
        specified_x_train.append(X_train[i])
        specified_y_train.append(2)
        
specified_x_train = np.array(specified_x_train)
specified_y_train = np.array(specified_y_train)

In [118]:
# Selecting classes in testing set
specified_x_test =  []
specified_y_test =  []

for i in range(len(y_test)):
    if (y_test[i] == 1)  :
        specified_x_test.append(X_test[i])
        specified_y_test.append(0)
    elif (y_test[i] == 3):
        specified_x_test.append(X_test[i])
        specified_y_test.append(1)
    elif (y_test[i] == 5):
        specified_x_test.append(X_test[i])
        specified_y_test.append(2)
        
specified_x_test = np.array(specified_x_test)
specified_y_test = np.array(specified_y_test)

In [119]:
# Transforming labels into one-hot encoding
specified_y_train = keras.utils.to_categorical(specified_y_train, num_classes)
specified_y_test = keras.utils.to_categorical(specified_y_test, num_classes)

In [120]:
# Manipulating Images
# * transforming to float point
# * interpolating image to 0-1 scale
specified_x_train = specified_x_train.astype('float32')
specified_x_test = specified_x_test.astype('float32')
specified_x_train /= 255
specified_x_test /= 255

## Data Modeling

In [111]:
# Model Architecture.

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=specified_x_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [112]:
# Checking model and it's overall params.
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_14 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 32, 30, 30)        9248      
_________________________________________________________________
activation_15 (Activation)   (None, 32, 30, 30)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 15, 15)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 32, 15, 15)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 64, 15, 15)        18496     
__________

### Fitting the model

In [126]:
# Fitting the model 
model.fit(specified_x_train, specified_y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(specified_x_test, specified_y_test),
              shuffle=True)

Train on 15000 samples, validate on 3000 samples
Epoch 1/10
15000/15000 [==============================] - 237s 16ms/step - loss: 0.5448 - acc: 0.7295 - val_loss: 0.5354 - val_acc: 0.7360

### Model Evaluation

In [128]:
scores = model.evaluate(specified_x_test, specified_y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

3000/3000 [==============================] - 21s 7ms/step